# Conversion of raw data in CSV format to MNE Raw

This notebook is used to convert CSV (from Brite) data into an MNE Raw object for further processing and analysis using the MNE-Python framework. It is used in conjunction with the `participants.xlsx` spreadsheet.

In [ ]:
import pandas as pd
import mne

## Define data file settings

Refer to the information in the Excel spreadsheet. Modify this section for the data file you want to convert. 

In [ ]:
participant_id = 'P17'
number_of_prompts = 11
brite_event = 826.82  # Time (in seconds) when cycling started (second event on Brite)

## Create MNE Raw object from CSV data

In [ ]:
# Read CSV data
data = pd.read_csv(f'data/csv/{participant_id}.csv')
data = data.transpose()

In [ ]:
ch_names = [
    'S1_D1 hbr',
    'S1_D1 hbo',
    'S2_D1 hbr',
    'S2_D1 hbo',
    'S3_D1 hbr',
    'S3_D1 hbo',
    'S1_D2 hbr',
    'S1_D2 hbo',
    'S3_D2 hbr',
    'S3_D2 hbo',
    'S4_D2 hbr',
    'S4_D2 hbo',
    'S2_D3 hbr',
    'S2_D3 hbo',
    'S3_D3 hbr',
    'S3_D3 hbo',
    'S5_D3 hbr',
    'S5_D3 hbo',
    'S6_D3 hbr',
    'S6_D3 hbo',
    'S3_D4 hbr',
    'S3_D4 hbo',
    'S4_D4 hbr',
    'S4_D4 hbo',
    'S5_D4 hbr',
    'S5_D4 hbo',
    'S8_D4 hbr',
    'S8_D4 hbo',
    'S5_D5 hbr',
    'S5_D5 hbo',
    'S6_D5 hbr',
    'S6_D5 hbo',
    'S7_D5 hbr',
    'S7_D5 hbo',
    'S4_D6 hbr',
    'S4_D6 hbo',
    'S8_D6 hbr',
    'S8_D6 hbo',
    'S9_D6 hbr',
    'S9_D6 hbo',
    'S5_D7 hbr',
    'S5_D7 hbo',
    'S7_D7 hbr',
    'S7_D7 hbo',
    'S8_D7 hbr',
    'S8_D7 hbo',
    'S10_D7 hbr',
    'S10_D7 hbo',
    'S8_D8 hbr',
    'S8_D8 hbo',
    'S9_D8 hbr',
    'S9_D8 hbo',
    'S10_D8 hbr',
    'S10_D8 hbo',
]

ch_types = [
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
    'hbr',
    'hbo',
]

sfreq = 50.0

In [ ]:
# Create MNE info object and load the data into a RawArray
info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)
raw = mne.io.RawArray(data, info, verbose=True)
raw.info['subject_info'] = {'his_id': participant_id}

In [ ]:
raw.info

## Add annotations

In [ ]:
# Define initial events:
# - Baseline starts 90 seconds before the brite_event.
# - Baseline ends / Cycling starts at the brite_event.

descriptions = ['baseline starts', 'baseline ends/cycling starts']
durations = [0, 0]
onsets = [brite_event-90, brite_event]

# Add "prompt" events. Each prompt is added every 180 seconds after the brite_event.
for i in range(number_of_prompts):
    descriptions.append('prompt')
    onsets.append(brite_event + (180*(i+1)) )
    durations.append(0)

# Apply the annotations to the raw object
raw.set_annotations(
    mne.Annotations(onset=onsets, duration=durations, description=descriptions)
)

## Set montage to raw object

In [ ]:
# Define channel positions in millimeters
ch_pos = {
    'D1': [39.56, 62.65, 34.63],
    'D2': [21.87, 73.35, 34.39],
    'D3': [23.22, 39.78, 71.38],
    'D4': [-1.09, 70.73, 47.82],
    'D5': [1.2, 31.87, 81.54],
    'D6': [-22.77, 75.39, 30.82],
    'D7': [-21.11, 43.18, 70.97],
    'D8': [-39.92, 62.32, 37.46],
    'S1': [36.67, 73.91, 5.64],
    'S2': [40.19, 41.9, 58.11],
    'S3': [20.15, 58.2, 57.59],
    'S4': [1.23, 83.42, 21.47],
    'S5': [-0.25, 54.13, 66.91],
    'S6': [-22.36, 19.57, 83.1],
    'S7': [20.49, 17.85, 83.74],
    'S8': [-19.1, 61.39, 55.58],
    'S9': [-35.32, 75.97, 4.81],
    'S10': [-38.58, 42.89, 60.36],
    'Fpz': [1.23, 83.42, 21.47],
    'Oz': [0.84, -112.65, 32.29],
    'T7': [-86.93, -22.91, 3.13],
    'T8': [84.91, -23.79, 3.42],
}

# Define the coordinate frame and fiducial points (in millimeters)
coord_frame = 'mni_tal'
nasion = [-0.84, 86.21, -57.16]
lpa = [-86.93, -22.91, 3.13]
rpa = [84.91, -23.79, 3.42]

# Convert all positions from millimeters to meters
for key, value in ch_pos.items():
    ch_pos[key] = [x / 1000 for x in value]
nasion = [x / 1000 for x in nasion]
lpa = [x / 1000 for x in lpa]
rpa = [x / 1000 for x in rpa]

# Create and set the montage (sensor configuration)
montage = mne.channels.make_dig_montage(
    ch_pos=ch_pos, nasion=nasion, lpa=lpa, rpa=rpa, coord_frame=coord_frame)
raw.set_montage(montage)

In [ ]:
# Define a sphere for sensor topography plotting (center and radius in meters)
sphere = (0, 0, 0, 0.09)

mne.viz.plot_sensors(raw.info, kind='topomap', sphere=sphere,
                     show_names=True, ch_type='hbr', verbose=True)
mne.viz.plot_sensors(raw.info, kind='3d', show_names=False, ch_type='hbo',
                     verbose=True, show=False).gca().view_init(azim=70, elev=15)

## Viewing raw data

In [ ]:
raw.info

In [ ]:
raw.plot()

## Save to file

In [ ]:
raw.save(
    fname=f'data/mne_raw/{participant_id}.fif',
    overwrite=True,
    verbose=True
)